### 测试 es.search和helpers.scan能否批量输出

In [3]:
'''
    连接es
'''
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch('http://ip:port')

doc_index = 'wechat_bl2'
doc_type = 'news'

query = {
  "from":0, "size":2,
  "query": {
    "multi_match": {
      "query": "互访",
      "fields": ["title^4", "content"],
      "analyzer": "ik_max_word"
    }
  },
  "_source": {
    "includes": "url"
  }
}

# es.search可以批量输出
# helpers.scan尽管设置了size，仍是全部输出

full_data2 = es.search(index=doc_index, doc_type=doc_type, body=query) 
# data = helpers.scan(es, query=query, index=doc_index, doc_type=doc_type, scroll="1m",timeout='1m')

In [8]:
print(full_data2["hits"]["hits"][0])

{'_index': 'wechat_bl2', '_type': 'news', '_id': 'MzA4NDA4OTgwMQ==_2650766157_1', '_score': 14.735005, '_source': {'url': 'http://mp.weixin.qq.com/s?__biz=MzA4NDA4OTgwMQ==&mid=2650766157&idx=1&sn=ff4238d496937defeda1aaafe25f3083&chksm=87e755ebb090dcfdf6bcef7254331149b9f7afd45d86d5bc0d86eb19843ffc8f4793b6db701f&scene=27'}}


### 测试将连接es返回的generator型结果，转成list，长久保存

In [1]:
'''
    从es里读取的content是html格式，需要解析成文本
'''
def sentencesMaker(html):
    sentences = []
    if not html or not html.strip():
        return sentences
    try:
        from html.parser import unescape
        html = unescape(html)

        import justext
        paragraphs = justext.justext(html, [])

        cache_sentences = ''

        for p in paragraphs:
            sent = p.text.strip().replace('\xa0', '').replace('\u3000', '')
            sent = sent.encode('gb2312', 'ignore').decode('gb2312').encode('gbk', 'ignore').decode('gbk')
            if not sent:
                continue

            # 可能是含有名字，需要进一步处理
            if len(cache_sentences) < 5:
                cache_sentences += ' ' + sent
            else:
                sentences.append(cache_sentences.strip())
                cache_sentences = sent

        if not not cache_sentences:
            sentences.append(cache_sentences.strip())
    except Exception as e:
        logger.error(e)

    return sentences

In [22]:
'''
    从36kr获取数据实验，36kr数据量较少，4933条左右
'''
# -*- coding: utf-8 -*-

from elasticsearch import helpers


def search_bl2(from_number, offsize):
    es_search_options = set_search_optional_bl2(from_number, offsize)
    es_result = get_search_result(es_search_options, index='36kr_bl2')
    return es_result

def search_ner(final_results):
    es_search_options = set_search_optional_ner(final_results)
    es_result = get_search_result(es_search_options, index='36kr_ner')
    return es_result

def get_id_list(es_result):
    final_result = []
    for item in es_result: # ["hits"]["hits"]
        final_result.append(item["_id"])
    return final_result

def get_cleaned_content_list(es_result):
    final_result = []
    for item in es_result:
        final_result.append("".join(sentencesMaker(item["_source"]["content"])).replace("\n", ""))
    return final_result


def get_search_result(es_search_options, index, scroll='5m', doc_type='news', timeout="1m"):
    es_result = helpers.scan(
        es,
        query=es_search_options,
        scroll=scroll,
        index=index,
        doc_type=doc_type,
        timeout=timeout
    )
    return es_result


def set_search_optional_bl2(from_number, offsize):
    # 检索选项
    es_search_options = {
      "query": {

        "match_all": {}
      },
        "_source":["content"]

    }
    return es_search_options

def set_search_optional_ner(final_results):
    # 检索选项
    es_search_options = {
          "query": {
            "ids":{
              "values": final_results
            }
          }, 
          "_source":  ["rel"]
    }
    return es_search_options


def split_samples(data):
    negative_samples = []
    positive_samples = []
    for i in data:
        if not i["_source"]["rel"] or len(i["_source"]["rel"]) < 2:
            negative_samples.append(i["_id"])
        else:
            positive_samples.append(i["_id"])
    return negative_samples, positive_samples


if __name__ == '__main__': 
    all_results_bl2_1 = search_bl2(0, 5)
    all_results_bl2_1 = [item for item in all_results_bl2_1]
    
#     j  = 0
#     for i in all_results_bl2_1:
#         j  += 1
#     print(j)
        
#     bl2_contents_results = get_cleaned_content_list(all_results_bl2_1)
#     all_results_bl2_1 = [item for item in all_results_bl2_1]
#     print(len(bl2_contents_results))
    
#     all_results_bl2_2 = search_bl2(0, 5)
    
#     k = 0
#     for i in all_results_bl2_2:
#         k += 1
#     print(k)
        
    bl2_ids_results = get_id_list(all_results_bl2_2)
    print(len(bl2_ids_results))
    
#     print("length of total samples from bl2:", len(bl2_ids_results))
    
#     ids_contents_dic = zip(bl2_ids_results, bl2_contents_results)
    
#     all_results_ner = search_ner(bl2_ids_results)
    
#     negative_samples, positive_samples = split_samples(all_results_ner)
    
#     print("length of total samples from ner:", len(negative_samples) + len(positive_samples))
#     print("length of negative samples:", len(negative_samples))
#     print("length of positive samples:", len(positive_samples))
    
#     with open("./toutiao_negative_samples.txt", "w", encoding="utf-8") as nf:
#         for _id, _content  in ids_contents_dic:
#             if _id in negative_samples:
#                 nf.write("1" + "\t" + _id + "\t" +_content + "\n")

In [28]:
bl2_ids_results = get_id_list(all_results_bl2_1)
print(len(bl2_ids_results))

4933


In [31]:
print(bl2_ids_results[0])

5116737


In [23]:
print(type(all_results_bl2_1))

<class 'list'>


In [26]:
print(len(all_results_bl2_1))

4933


In [24]:
print(all_results_bl2_1[0])

{'_index': '36kr_bl2', '_type': 'news', '_id': '5116737', '_score': None, '_source': {'content': '<figure><img src="https://pic.36krcnd.com/201801/30011449/rrt6lha92dha3p38!heading"></figure>\n<p>以比特币为代表的区块链1.0时代，解决了点对点价值传输中的信任问题，但比特币逐渐成为了“投资品”，其流通和支付的属性越来越弱；到了区块链2.0时代，以太坊通过智能合约实现了不同场景的应用，但共用一条主链容易造成网络拥堵，甚至瘫痪。<br/></p><p>在底层设计上，比特币和以太坊分别对应确定有限和确定增量的区块，两者的“过度共识”无法承载社会规模的生产。</p><p>如果把比特币比作“一个点”、以太坊比作“一条线”，那么黑派科技想站在构建完整生态的角度设计“一个面”。黑派科技于去年11月成为新加坡ValueCyber中国区技术合作伙伴，目前正在开发的区块链3.0项目ValueCyber，<strong>为“去中心化”的应用和应用间的协作提供工具和平台。</strong></p><p><strong>ValueCyber通过“价值-债务网络”的集体共识机制，实现多链、多系统、多场景的互联；同时根据接入的生产网络的流动性需求，自动调整token总量，保持货币的“弹性”与价值的相对恒定。</strong></p><p>在技术方面，ValueCyber采用分层结构解决“过度共识”的问题，即允许每个应用根据自身需求，形成独立的链，并连接到ValueCyber主链。当单一应用需要“强共识”时，可与外界跨链协作，在大部分不需要跨链的时间，只需链内解决问题。</p><p>ValueCyber结合了以太坊中智能合约的思想，以及比特币区块链的安全性（防止算力攻击），同时向上开放满足通用性，向下兼容主流公有链。开发者可直接接入ValueCyber的跨链协议，降低迁移成本。</p><p><span><img alt="区块链应用 | \u200b将区块链技术映射到实体经济？「ValueCyber」想成为下一代区块链底层" data-img-size-val="916,449" data-src="htt

### 按主题词从es里重新获取wechat数据，划分正负样本

In [1]:
topic_words = ["互访", "沙龙", "博览会", "展览会", "圆桌", "挂牌仪式", "发布会", "演讲", "组委会", "对接会", "会议", "理事会", 
              "报告", "分享", "入选", "培训", "融资", "大会", "峰会", "年会", "高峰论坛", "论坛", "研讨会", "比赛", "大赛"]

In [2]:
len(topic_words)

25

In [124]:
'''
    从wechat取正、负样本
'''
# -*- coding: utf-8 -*-

from elasticsearch import helpers


def search_bl2(from_number, off_size):
    es_search_options = set_search_optional_bl2(from_number, off_size)
    es_result = get_search_result(es_search_options, index='wechat_bl2')
    return es_result

def search_ner(final_results, from_number, off_size):
    es_search_options = set_search_optional_ner(final_results, from_number, off_size)
    es_result = get_search_result(es_search_options, index='wechat_ner')
    return es_result

def get_id_list(es_result):
    final_result = []
    for item in es_result: # ["hits"]["hits"]
        final_result.append(item["_id"])
    return final_result

# def get_cleaned_content_list(es_result):
#     final_result = []
#     for item in es_result:
#         final_result.append("".join(sentencesMaker(item["_source"]["content"])).replace("\n", ""))
#     return final_result


def get_search_result(es_search_options, index, scroll='5m', doc_type='news', timeout="1m"):
    es_result = helpers.scan(
        es,
        query=es_search_options,
        scroll=scroll,
        index=index,
        doc_type=doc_type,
        timeout=timeout
    )
    return es_result


def set_search_optional_bl2(from_number, off_size):
    # 检索选项
    es_search_options = {
        "from":from_number, "size":off_size,
        "query": {
            "analyzer": "ik_max_word"
            "bool": {
              "should": [
                {"terms": topic_words},
                {"terms": topic_words},
              ]
           }
        },
        "_source":["content"]
    
    }
    return es_search_options

def set_search_optional_ner(final_results, from_number, off_size):
    # 检索选项
    es_search_options = {
        "from":from_number, "size":off_size,
          "query": {
            "ids":{
              "values": final_results
            }
          }, 
          "_source":  ["rel"]
    }
    return es_search_options


def split_samples(data):
    negative_samples = []
    positive_samples = []
    for i in data:
        if not i["_source"]["rel"] or len(i["_source"]["rel"]) < 3:
            negative_samples.append(i["_id"])
        else:
            positive_samples.append(i["_id"])
    return negative_samples, positive_samples


if __name__ == '__main__': 
    all_results_bl2 = search_bl2(0,5)
    all_results_bl2 = [item for item in all_results_bl2]  # 把从bl2获取的数据保存成list，数据里包括id,content等较少的信息
    print("all_results_bl2长度：", len(all_results_bl2))
    
    bl2_ids_results = get_id_list(all_results_bl2)  # 接着返回的bl2结果中，取出来ids，传给ner返回rel
    print("从bl2获取的id的长度：", len(bl2_ids_results))
    
    all_results_ner = search_ner(bl2_ids_results, 0, 5)   # ner利用bl2返回的ids的数据，取rel
    all_results_ner = [item for item in all_results_ner]  # 把从ner获取的结果也保存成list，包括id，rel等较少的信息
    print("all_results_ner长度：", len(all_results_ner))
    
    
    negative_samples, positive_samples = split_samples(all_results_ner)  # 查看ner返回的rel信息，划分正负样本的id
    
    print("length of total samples from ner:", len(negative_samples) + len(positive_samples))
    print("length of negative samples:", len(negative_samples))
    print("length of positive samples:", len(positive_samples))
    
    # 根据正负样本的id，去bl2返回的结果中，取回相应的content
    with open("./wechat_negative_samples.txt", "a", encoding="utf-8") as nf, open("./wechat_positive_samples.txt", "a", encoding="utf-8") as pf:  
        for item in all_results_bl2:
            if item["_id"] in negative_samples:
                nf.write("1" + "\t" + item["_id"] + "\t" + "".join(sentencesMaker(item["_source"]["content"])).replace("\n", "") + "\n")
            if item["_id"] in positive_samples:
                pf.write("0" + "\t" + item["_id"] + "\t" + "".join(sentencesMaker(item["_source"]["content"])).replace("\n", "") + "\n")

all_results_bl2长度： 3184
从bl2获取的id的长度： 3184
all_results_ner长度： 3183
length of total samples from ner: 3183
length of negative samples: 2357
length of positive samples: 826


In [125]:
# sum_negative_samples, sum_positive_samples, total_samples = 0, 0, 0
sum_negative_samples += len(negative_samples)
sum_positive_samples += len(positive_samples)
total_samples = sum_negative_samples + sum_positive_samples
print("所有总样本个数%d" % total_samples)
print("所有负样本个数%d，占比%.2f" % (sum_negative_samples, (sum_negative_samples / total_samples)))
print("所有正样本个数%d，占比%.2f" % (sum_positive_samples, (sum_positive_samples / total_samples)))

所有总样本个数184438
所有负样本个数128185，占比0.70
所有正样本个数56253，占比0.30


In [128]:
'''
    剔除结果中重复的文章
'''
with open("./filtered_wechat_negative_samples.txt", "w", encoding="utf-8") as fnf, open("./filtered_wechat_positive_samples.txt", "w", encoding="utf-8") as fpf:
    with open("./wechat_negative_samples.txt", "r", encoding="utf-8") as nf, open("./wechat_positive_samples.txt", "r", encoding="utf-8") as pf:

        nf_data = nf.readlines()
        print("negative_samples中所有文件长度：", len(nf_data))
        print("filtered_negative_samples长度：", len(set(nf_data)))
        count1 = 0
        for i in set(nf_data):
            if len(i.strip().split("\t")) == 3:
                fnf.write(i)
                count1 += 1
        print("负样本剔除空内容之后文件长度：", count1)
        print("\n")
        
        pf_data = pf.readlines()
        print("positive_samples中所有文件长度：", len(pf_data))
        print("filtered_positive_samples长度：", len(set(pf_data)))
        count2 = 0
        for j in set(pf_data):
            if len(j.strip().split("\t")) == 3:
                fpf.write(j)
                count2 += 1
        print("正样本剔除空内容之后文件长度：", count2)
        print("\n")
        
        print("包含空样本的负样本占比：%.2f" %  (len(set(nf_data))/ (len(set(pf_data)) + len(set(nf_data)))))
        print("包含空样本的正样本占比：%.2f" %  (len(set(pf_data))/ (len(set(pf_data)) + len(set(nf_data)))))
        print("\n")
        
        print("实际负样本占比：%.2f" % (count1/(count1+count2)))
        print("实际正样本占比：%.2f" % (count2/(count1+count2)))

negative_samples中所有文件长度： 128185
filtered_negative_samples长度： 47386
负样本剔除空内容之后文件长度： 47382


positive_samples中所有文件长度： 56253
filtered_positive_samples长度： 12738
正样本剔除空内容之后文件长度： 12738


包含空样本的负样本占比0.79：
包含空样本的正样本占比0.21：


实际负样本占比0.79
实际正样本占比0.21


### 从wechat获取的正负样本，正样本保留，负样筛选，保留四分之一

In [129]:
select_number = []
for i in range(0, 47382, 4):
    select_number.append(i)
print(len(select_number))

11846


In [130]:
'''
    选大概四分之一的负样本保留下来，并剔除content为空的数据
'''
with open("./filtered_wechat_negative_samples.txt", "r", encoding="utf-8") as outf:
    with open("./selected_wechat_negative_samples.txt", "w", encoding="utf-8") as inf:
        data = outf.readlines()
        count = 0
        for i in select_number:
            if len(data[i].strip().split("\t")) == 3: # 判断content是否为空
                inf.write(data[i])
                count += 1
        print(count)

11846
